### Imports

In [7]:
import numpy as np
import pandas as pd
import re
import warnings
import datetime
warnings.filterwarnings("ignore")
import pickle
import scipy
from scipy import signal

The following functions are in the order that they are intended to be run. 

In [1]:
#finds groups given array
from itertools import groupby
from collections import defaultdict

def find_groups(arr):
    i = 0
    result = defaultdict(list)
    for k, g in groupby(arr):
        elems = len(list(g))
        result[k].append((i, i+elems-1))
        i += elems
    
    return result

In [6]:
#median filter on normalized OF
def median_filter(video_name, filter_length = 5):
    ref_df = df[df['video_filename'] == video_name]
    return scipy.signal.medfilt(ref_df['percentile_normalized_flow'].values, filter_length)

In [5]:
#is in high activity
def is_in_hact(threshold_value, median_filter):
    is_in = []
    for i in range(len(median_filter)):
        if median_filter[i] >= threshold_value:
            is_in.append(True) 
        else:
            is_in.append(False)
    return is_in

In [6]:
#calculates the flicker interval
def calculate_frames(start, end, interval):
    mid_frame = (int)((end + start) / 2)
    start_frame = mid_frame - interval
    end_frame = mid_frame + interval
    
    #based off the script Doran wrote in video_clipper.ipynb under the video_analysis directory
    if end_frame > 54000 or start_frame < 0 or end_frame < 0 or start_frame > 54000:
        return None
    
    return start_frame, end_frame

In [1]:
#gets column information from specified flicker interval
def annotations(video_name, video_df, flicker, interval, col):
    flicker_start = flicker[0]
    flicker_end = flicker[1]
    
    frame_start, frame_end = calculate_frames(flicker_start, flicker_end, interval)
    
    name = video_name.split(".")[0] + "_f" + format(flicker_start) + "-" + "f" + format(flicker_end) + ".mp4"
    ants = video_df[col][frame_start:frame_end].values
    
    return name, ants

In [7]:
#plots flickers of given video
def plt_video(video_name, flickers, interval, info):
    video_df = mod_df.loc[mod_df["video_filename"] == video_name]
    every_flicker = flickers.get(video_name)
    
    if every_flicker != None:
        for i, flicker in enumerate(every_flicker):
            start, end = calculate_frames(flicker[0], flicker[1], interval)
            name, ant = annotations(video_name, video_df, flicker, interval, info)
            print(name)

            x = range(len(ant))
            plt.plot(x, ant)
            plt.plot(x, video_df["median_filter_pnf"][start:end].values)
            plt.axhline(y=0.35, color='r', linestyle='-')
            plt.title(name)
            plt.show()
    else:
        print(video_name)